# Лекция №6. Функциональное программирование

П.Н. Советов, РТУ МИРЭА

## Введение

До сих пор мы, в основном, явно указывали в программах на Питоне последовательность действий, содержащих присваивания и изменяемые данные. Это соответствует императивному подходу к программированию.

В функциональном программировании, в противовес программированию императивному, основной операцией является применение функций к своим аргументам. Такие функции являются "чистыми" в математическом смысле, то есть результат их выполнения полностью определяется входными аргументами. В противоположность чистым функциям, вычисление функции, обладающей побочным эффектом, зависит неявным образом от окружения. Например, функция random из одноименного модуля стандартной библиотеки Питона использует в своей работе глобальную переменную.

При анализе императивной программы программисту приходится в уме пошагово имитировать работу компьютера. В функциональном программировании используется более простая, чем в императивном программировании, подстановочная модель вычислений (выражения заменяются их результатами), которой программист может пользоваться умозрительно с большим удобством.

Важным свойством программ, которое обеспечивается функциональным программированием, является компонуемость или сочетаемость — сложные программы строятся, свободно используя простые строительные блоки-функции, в духе блоков игрового конструктора. В Питоне компонуемость не всегда выдерживается. К примеру, мы не можем поместить обычный for или if внутрь некоторого выражения. Конечно, мы могли бы использовать списковое включение или тернарный if, но эти конструкции обладают рядом ограничений, то есть плохо компонуются.

Остается вопрос, каким образом функциональные программы, без присваиваний и изменяемых данных, могут делать что-то полезное. При использовании функционального стиля программирования в таком языке, как Питон, предпочтительной является типовая архитектура системы под названием "функциональное ядро и императивная оболочка" (Functional core, imperative shell):

1. Ключевые вычислительные задачи и задачи преобразования данных формируют функциональное ядро системы. Здесь применяются только чистые функции и неизменяемые типы данных.
1. Работа с внешней средой, ввод/вывод, вспомогательные задачи организуются с помощью императивной оболочки.

## Еще раз о пространствах имен и функциях

В Питоне используется несколько пространств имен, вложенных друг в друга:

1. Пространство имен модуля (globals), которое включает в себя пространство имен встроенных функций (builtins).
1. Пространство имен объемлющих функций (nonlocal).
1. Пространство имен текущей функции (locals).

Переменная в функции может относиться только к одному из этих пространств имен. Рассмотрим следующий пример:

In [1]:
a = 42

def f():
    print(a)
    a = 1
    
# Раскомментируйте, чтобы получить ошибку
# f()

Компилятор Питона при трансляции функции руководствуется следующими правилами:
1. Если `a` не используется в левой части присваивания, а также отсутствует в объемлющих функциях, то `a` трактуется, как элемент из globals.
1. Если `a` не используется в левой части присваивания, но присутствует в объемлющей функции, то формируется nonlocal-ссылка на `a` из объемлющей функции.
1. Если `a` присваивается некоторое значение, то `a` является элементом locals.

В примере выше `a` трактуется, как локальная переменная, поэтому вызов print до присваивания приводит к ошибке. Разумеется, с помощью global или nonlocal (о нем будет сказано далее) можно явно указать, к какому пространству имен принадлежит `a`:

In [2]:
a = 42

def f():
    global a
    print(a)
    a = 1

f()

42


## Функции высшего порядка

В Питоне функции являются объектами первого класса. Это означает, что функции можно использовать, как аргументы, возвращать из функций, а также записывать в переменные. Это позволяет использовать важнейший элемент функционального программирования — функции высшего порядка.

Функция высшего порядка (ФВП), помимо значений других типов, использует функции в качестве своих аргументов/результата. В математике известны примеры ФВП. Это, например, производные и интегралы. В современном программировании без ФВП тоже трудно обойтись: даже на языке C стандартная функция сортировки qsort требует ссылки на функцию в качестве одного из своих аргументов.

Ниже показан пример ФВП, которая принимает на вход функцию:

In [3]:
def f(func, arg):
    return func(arg)
    
def incr(x):
    return x + 1
    
print(f(incr, 10))

11


Пример ФВП, возвращающей функцию:

In [4]:
def f(x):
    def func(y):
        return y * x
    return func

g = f(10)
[g(x) for x in range(10)]

[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]

Наконец, пример ФВП, получающей и возвращающей функцию:

In [5]:
def f(g, x):
    def func(size):
        return ' '.join([g(x) for i in range(size)])
    return func

def f1(x):
    return '*' * x

f2 = f(f1, 2)
f2(10)

'** ** ** ** ** ** ** ** ** **'

### Замыкания

Мы уже видели, что вложенные функции в Питоне могут использовать не только аргументы и локальные переменные, но также и свободные переменные — имена из объемлющих функций. Это возможно благодаря механизму лексического замыкания, при котором формируется специальный объект, включающий себя как ссылку на функцию, так и данные, которые функция использует из объемлющих функций.

Рассмотрим следующий пример:

In [6]:
def make_adder(x):
    def adder(y):
        # Замыкание, включающее в себя свободную переменную x
        return x + y
    return adder
    
f = make_adder(10)
print(f(5))

# Переменные adder
print(f.__code__.co_varnames)
# Свободные переменные adder
print(f.__code__.co_freevars)
# Значение x из захваченного окружения
print(f.__closure__[0].cell_contents)

15
('y',)
('x',)
10


Для модификации захваченной свободной переменной необходимо явно указать с помощью ключевого слова nonlocal, что модифицируемая переменная не является локальной или глобальной, а относится к одной из объемлющих функций:

In [7]:
def make_counter(n):
    i = 0
    def counter():
        nonlocal i # Необходимое указание, поскольку далее используется присваивание
        i = i - 1 if i > 0 else n
        return i
    return counter

counter = make_counter(2)

for i in range(16):
    print(counter(), end=' ')

2 1 0 2 1 0 2 1 0 2 1 0 2 1 0 2 

### Анонимные функции

Функциональный стиль программирования предполагает создание большого числа ФВП. При этом часто приходится создавать небольшие функции-аргументы, нужные только для одной сиюминутной задачи. В подобных случаях удобнее воспользоваться механизмом создания анонимных функций с помощью ключевого слова lambda.

Определение

```Python
func = lambda аргументы: выражение
```

эквивалентно следующему:

```Python
def func(аргументы):
    return выражение
```

Лямбда-выражение можно передать на вход или возвратить из ФВП, как показано ниже:

In [8]:
def f(g, x):
    return lambda size: ' '.join([g(x) for i in range(size)])

f(lambda x: '*' * x, 2)(10)

'** ** ** ** ** ** ** ** ** **'

Еще один пример:

In [9]:
def twice(f):
    def g(x):
        return f(f(x))
    return g

twice(twice(lambda x: x * x))(2)

65536

Обратите внимание на ограничения lambda. В ее теле могут использоваться только те конструкции, которые доступны в качестве выражения, возвращаемого с помощью return.

Классическими в функциональном программировании являются функции map, filter и reduce. Они реализованы в Питоне, но для целей более близкого знакомства с ФВП далее приводится упрощенная реализация каждой их них.

Функция map принимает на вход унарную функцию и список, а возвращает новый список, элементы которого являются результатами применения функции-аргумента к элементам исходного списка:

In [10]:
def my_map(func, lst):
    res = []
    for x in lst:
        res.append(func(x))
    return res

my_map(lambda x: x * x, range(10))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Функция filter принимает на вход унарный предикат и список, а возвращает новый список, элементы которого удовлетворяют предикату:

In [11]:
def my_filter(pred, lst):
    res = []
    for x in lst:
        if pred(x):
            res.append(x)
    return res

my_filter(lambda x: x % 2, range(10))

[1, 3, 5, 7, 9]

Функция reduce принимает на вход унарную функцию и список, последовательно применяет функцию-аргумент, сводя список до единственного значения:

In [12]:
def my_reduce(func, lst):
    x = lst[0]
    for y in lst[1:]:
        x = func(x, y)
    return x

my_reduce(lambda x, y: x + y, range(10))

45

Вместо функций map и filter в Питоне предпочтительнее использовать уже знакомые нам списковые включения, а также генераторы, о которых будет говориться далее.

### Сопоставление с образцом

Начиная с версии 3.10 в Питоне существует конструкция сопоставления с образцом match/case. Ее можно рассматривать, как аналог switch/case из других языков, но главное достоинство match/case состоит в связывании отдельных элементов составных шаблонов с переменными.

Общая схема match/case выглядит следующим образом:

```
match <вычисляемое выражение>
    case <шаблон_1>:
        блок
    case <шаблон_2>:
        ...
    case <шаблон_n>:
        блок
```

В качестве шаблонов могут выступать составные данные различных типов. В примере ниже используются классы:

In [2]:
class List:
    def __init__(self, data, tail=None):
        self.data = data
        self.tail = tail


# Поиск по ключу в связном списке, хранящему пары (ключ, значение)
def lookup(lst, key):
    match lst:
        case List(data=(k, v)) if k == key:
            return v
        case List(tail=t):
            return lookup(t, key)


lst = List(('a', 1), List(('b', 2), List(('c', 3))))


print(lookup(lst, 'a'))
print(lookup(lst, 'b'))
print(lookup(lst, 'c'))
print(lookup(lst, 'z'))

1
2
3
None


Дополнительная информация о match/case приведена в [документации](https://peps.python.org/pep-0636/).

### Декораторы

Декораторы синтаксически представляют собой аннотации, которые указываются перед аннотируемой функцией. Реализуются они с помощью ФВП, которая принимает на вход функцию и возвращает функцию.

Декоратор функции выглядит следующим образом:

```Python
@decorator
def f():
    print('...')
```    

Что эквивалентно выражению

```Python
def f():
    print('...')

f = decorator(f)
```

Простой пример:

In [13]:
def decorator(f):
    print('Декоратор сработал.')
    return f

@decorator
def f():
    print('...')

print('Запуск функции.')
f()

Декоратор сработал.
Запуск функции.
...


Еще один пример, с использованием ранее определенной функции twice, уже готовой к использованию в качестве декоратора:

In [14]:
@twice
def f(x):
    return x * x

f(f(2))

65536

Пример с регистрацией вызовов функции:

In [15]:
def call_tracer(f):
    def g(x):
        print('Запуск функции {}({})'.format(f.__name__, x))
        return f(x)
    return g

@call_tracer
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)

fib(4)

Запуск функции fib(4)
Запуск функции fib(3)
Запуск функции fib(2)
Запуск функции fib(1)
Запуск функции fib(0)
Запуск функции fib(1)
Запуск функции fib(2)
Запуск функции fib(1)
Запуск функции fib(0)


3

Декораторы могут иметь параметры, как показано в примере ниже:

In [16]:
def decorator(x):
    def func(f):
        print(x)
        return f
    return func
    
@decorator('Запуск')
def my_func(x):
    print(x)
    
my_func(10)

Запуск
10


Помимо декораторов функции существуют также декораторы класса, которые принимают на вход класс и возвращают класс.

## Генераторы

Генераторы представляют собой особый вид функций, внутри которых находится оператор yield. Этот оператор приостанавливает вычисления внутри функции, сохраняет текущее состояние этих вычислений и, аналогично return, осуществляет возврат в вызывающую функцию. При этом вычисления в генераторе можно возобновить сразу после последнего вызова yield.

```Python
def func():
    ...
    yield выражение
    ...
```

Вызов функции-генератора осуществляется особым способом, с помощью функции next, как показано в примере ниже:

In [17]:
def countup(n):
    print('Начало цикла:')
    for i in range(n):
        yield i
        print('Следующая итерация:')

# Создан объект генератора, ему передан аргумент,
# но countup еще не начал работу
c = countup(4)
print(type(c))
for i in range(4):
    print(next(c)) # выполнение останавливается после yield i
    print('...')
try:
    print(next(c))
except StopIteration:
    print('Выход из генератора по исключению')


<class 'generator'>
Начало цикла:
0
...
Следующая итерация:
1
...
Следующая итерация:
2
...
Следующая итерация:
3
...
Следующая итерация:
Выход из генератора по исключению


Генераторы поддерживают протокол итераторов:

In [18]:
for i in countup(4):
    print(i, end=' ')

Начало цикла:
0 Следующая итерация:
1 Следующая итерация:
2 Следующая итерация:
3 Следующая итерация:


In [19]:
def gen():
    for i in range(10):
        yield i

list(gen())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

В следующем примере генерируются все возможные пароли, символы которых принадлежат некоторому алфавиту. Использование генератора позволяет упростить код и сделать его производительным, поскольку порождение паролей происходит по необходимости, без создания лишних структур данных.

In [20]:
def gen_password(abc):
    for a in abc:
        for b in abc:
            for c in abc:
                for d in abc:
                    yield f'{a}{b}{c}{d}'


for p in gen_password('absp'):
    if p == 'pass':
        print('Пароль найден')
        break

Пароль найден


Предположим, что мы хотим реализовать следующие вычисления:

In [21]:
import math

my_filter(lambda x: x >= 0, my_map(math.sin, list(range(10000))))[:10]

[0.0,
 0.8414709848078965,
 0.9092974268256817,
 0.1411200080598672,
 0.6569865987187891,
 0.9893582466233818,
 0.4121184852417566,
 0.4201670368266409,
 0.9906073556948704,
 0.6502878401571168]

Здесь формируется большое количество промежуточных данных, от которых можно избавиться, если реализовать вычисления "по необходимости", с помощью генераторов.

Ниже показан более эффективный вариант решения той же задачи с помощью генераторов:

In [22]:
# Сгенерировать бесконечный поток чисел 0, 1, 2, ...
def gen_count():
    i = 0
    while True:
        yield i
        i += 1

def gen_map(func, gen):
    for x in gen:
        yield func(x)

def gen_filter(pred, gen):
    for x in gen:
        if pred(x):
            yield x

# Возвратить список n первых сгенерированных элементов
def take(n, gen):
    return [next(gen) for i in range(n)]

take(10, gen_filter(lambda x: x >= 0, gen_map(math.sin, gen_count())))

[0.0,
 0.8414709848078965,
 0.9092974268256817,
 0.1411200080598672,
 0.6569865987187891,
 0.9893582466233818,
 0.4121184852417566,
 0.4201670368266409,
 0.9906073556948704,
 0.6502878401571168]

В Питоне существуют генераторы-выражения, которые действуют аналогично уже изученным нами списковым включениям, но явно не порождают список. Генераторы-выражения представляют собой простой синтаксис для описания небольших генераторов:

In [23]:
lst = [i * i for i in range(10)]
gen = (i * i for i in range(10)) # Генератор-выражение
print(lst, gen, list(gen))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81] <generator object <genexpr> at 0x0000018078F4EDD0> [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]


In [24]:
print(sum((i ** 3 for i in range(10))))
print(sum(i ** 3 for i in range(10))) # Скобки необязательны

2025
2025


## Неизменяемые типы данных

В Питоне существует аналог изменяемого массива — уже хорошо знакомый нам кортеж. Действительно, элементы кортежа нельзя изменить:

In [25]:
t1 = (1, 2, 3)
t2 = ('a', ['b', 'c'])
# t1[2] = 42 Не сработает
t2[1][0] = 50 # Сработает, поскольку ссылка на список не изменилась
print(t2)

def replace(tup, index, val):
    '''
    Замена элемента кортежа в функциональном стиле.
    Возвращается новый кортеж.
    '''
    return tup[:index] + (val,) + tup[index + 1:]

replace(t1, 2, 42)

('a', [50, 'c'])


(1, 2, 42)

Множество в Питоне является изменяемой структурой данных, поэтому не может, в частности, выступать в качестве ключа словаря. Имеется неизменяемый вариант множества — frozenset:

In [26]:
s1 = set(['a', 'b', 'c'])
d = {}
# d[s1] = 42 Не сработает
s2 = frozenset(['a', 'b', 'c'])
d[s2] = 42
d

{frozenset({'a', 'b', 'c'}): 42}

В качестве неполного аналога словаря в Питоне выступает структура данных namedtuple (именованный кортеж):

In [27]:
from collections import namedtuple

Person = namedtuple('Person', 'name age')
p1 = Person(name='Ivan', age=20)
print(p1.name, p1.age)
# p1.age += 1 Не сработает
print(p1._replace(age=p1.age + 1)) # Возвращен новый именованный кортеж

Ivan 20
Person(name='Ivan', age=21)


## Модуль functools

Модуль functools содержит ряд полезных определений, которые используются для функций высшего порядка. В частности, там имеется уже рассмотренная выше функция reduce.

Декоратор lru_cache осуществляет запоминание (мемоизацию) вычислений чистой функции. Если функция была вызвана повторно с теми же аргументами, то результат будет взят из таблицы:

In [28]:
from timeit import timeit
from functools import lru_cache

fact1 = lambda n: n * fact1(n - 1) if n else 1

@lru_cache
def fact2(n):
    return n * fact2(n - 1) if n else 1

print(timeit(lambda: fact1(1000), number=1000))
print(timeit(lambda: fact2(1000), number=1000))

0.5109138
0.0013229000000003488


Функция partial принимает на вход функцию и возвращает новую функцию, некоторые аргументы которой уже содержат конкретные значения:

In [29]:
from functools import partial

def power(x, n):
    return x ** n

power2 = partial(power, n=2)
# Эквивалентно power2 = lambda x: power(x, n=2)

print(power2(10))

100


## Модуль itertools

Модуль itertools предназначен для работы с итераторами (генераторами) в высокопроизводительном функциональном стиле.

Итератор chain группирует итераторы-аргументы для последовательного выполнения:

In [30]:
from itertools import chain

for i in chain(range(1, 5), "abcd", range(10, 15)):
    print(i, end=' ')

1 2 3 4 a b c d 10 11 12 13 14 

В itertools имеется ряд итераторов для операций из области комбинаторики:

In [31]:
from itertools import product

for i in product('abcd', repeat=2):
    print(i)

('a', 'a')
('a', 'b')
('a', 'c')
('a', 'd')
('b', 'a')
('b', 'b')
('b', 'c')
('b', 'd')
('c', 'a')
('c', 'b')
('c', 'c')
('c', 'd')
('d', 'a')
('d', 'b')
('d', 'c')
('d', 'd')


In [32]:
from itertools import combinations

for i in combinations('abcd', 3):
    print(i)

('a', 'b', 'c')
('a', 'b', 'd')
('a', 'c', 'd')
('b', 'c', 'd')


In [33]:
from itertools import permutations

for i in permutations('abcd', 2):
    print(i)

('a', 'b')
('a', 'c')
('a', 'd')
('b', 'a')
('b', 'c')
('b', 'd')
('c', 'a')
('c', 'b')
('c', 'd')
('d', 'a')
('d', 'b')
('d', 'c')


Итератор count является расширенной версией реализованной выше функции gen_count, а islice позволяет сделать срез для итератора:

In [34]:
from itertools import count, islice

list(islice((i for i in count(5)), 0, 10))

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

Итератор groupby последовательно объединяет в группы элементы итератора-аргумента по заданному ключу.

In [35]:
from itertools import groupby

# Ключом по умолчанию является сам элемент
print([(k, len(list(g))) for k, g in groupby('AAAABBBCCDAABBB')])

# Ключом является len
print([(k, list(g)) for k, g in groupby(['one', 'one', 'two', '3', '3', '3'], key=len)])

[('A', 4), ('B', 3), ('C', 2), ('D', 1), ('A', 2), ('B', 3)]
[(3, ['one', 'one', 'two']), (1, ['3', '3', '3'])]


## Библиотека NumPy

In [36]:
import numpy as np

Библиотека NumPy предназначена для вычислений на многомерных массивах. Она используется в качестве базовой в большом количестве библиотек для научных вычислений, в том числе для анализа данных и машинного обучения.

Можно выделить следующие характерные черты NumPy:

* Использование типизированных, однородных n-мерных массивов фиксированной длины.
* Операции применяются сразу ко многим элементам массивов.
* Поддерживается векторизация, что позволяет повысить производительность вычислений.

Во многих случаях код на Питоне можно ускорить, сократив его, то есть избавившись от низкоуровневых операций. Это верно и для NumPy. Компактный код вычислений на NumPy, в котором, в частности, отсутствуют циклы, может оказаться производительнее кода на обычном Питоне в десятки и даже сотни раз.

In [37]:
from timeit import timeit

x, y = list(range(0, 1000)), list(range(0, 1000))

def test1(x, y):
    return [a * b for a, b in zip(x, y)]

nx, ny = np.array(x), np.array(y)

def test2(x, y): return x * y

t1 = timeit(lambda: test1(x, y), number=1000)
t2 = timeit(lambda: test2(nx, ny), number=1000)
t1 / t2

36.632734722624065

### Свойства массива

In [38]:
a = np.array([[1, 2, 3], [4, 5, 6]])

# Тип
print(type(a))

# Размерность
print(a.ndim)

# Форма
print(a.shape)

# Количество элементов
print(a.size, a.nbytes)

# Тип элементов
print(a.dtype, a.itemsize)

<class 'numpy.ndarray'>
2
(2, 3)
6 24
int32 4


### Функции создания массива

In [39]:
# Создание на основе Python-данных
py_a = (1, 2, 3, 4)
print(np.array(py_a))

# Заполнение нулями
print(np.zeros(shape=(2, 2), dtype='int8'))

# Заполнение единицами
print(np.ones(shape=(4,), dtype='float'))

# Заполнение значением
print(np.full((3,), 42))

# Без инициализации
print(np.empty((3,)))

# Интервал значений
print(np.arange(1, 10))
print(np.arange(0, 1, step=0.1))

# Диагональная матрица
print(np.identity(4, dtype=int))

# Случайные значения
print(np.random.randint(0, 255, (4,)))

[1 2 3 4]
[[0 0]
 [0 0]]
[1. 1. 1. 1.]
[42 42 42]
[ 2.70972276e-306  4.96759808e-310 -9.63197469e-196]
[1 2 3 4 5 6 7 8 9]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
[[1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]
[215  91 125 155]


### Операции над массивами

Форма массива может быть изменена. Это не приводит к изменению самих данных массива:

In [40]:
a = np.arange(8)
print(a)
print(a.reshape(2, 4))
print(a.reshape(4, 2))
print(a.reshape(2, 2, 2))

[0 1 2 3 4 5 6 7]
[[0 1 2 3]
 [4 5 6 7]]
[[0 1]
 [2 3]
 [4 5]
 [6 7]]
[[[0 1]
  [2 3]]

 [[4 5]
  [6 7]]]


Транспонирование также не приводит к изменению данных:

In [41]:
print(a.reshape(2, 4))
print(a.reshape(2, 4).T)

[[0 1 2 3]
 [4 5 6 7]]
[[0 4]
 [1 5]
 [2 6]
 [3 7]]


Операции выполняются поэлементно:

In [42]:
a = np.array([1, 2, 3, 4])
b = np.array([2, 3, 4, 5])

print(a * 10)
print(a + b)
print(a * b)
print(a ** b)
print(np.sin(a))

print(np.negative(a))
print(a > 2)

[10 20 30 40]
[3 5 7 9]
[ 2  6 12 20]
[   1    8   81 1024]
[ 0.84147098  0.90929743  0.14112001 -0.7568025 ]
[-1 -2 -3 -4]
[False False  True  True]


In [43]:
print(a.sum())
print(a.prod())
print(a.max())
print(a.min())

print(a.reshape(2, 2))
print(a.reshape(2, 2).min(axis=1))
print(a.reshape(2, 2).max(axis=1))

10
24
4
1
[[1 2]
 [3 4]]
[1 3]
[2 4]


В общем случае можно использовать операции свертки в духе reduce() из Питона:

In [44]:
print(np.add.reduce(a))
print(np.multiply.reduce(a))

10
24


Функция where(массив_условий, массив_при_true, массив_при_false):

In [45]:
print(a)
cond = np.arange(a.size) % 2 == 0
print(cond)
print(np.where(cond, a, 0))

[1 2 3 4]
[ True False  True False]
[1 0 3 0]


### Способы индексации массивов

Простая индексация:

In [46]:
print(a[0], a[3])


# Запись производится по ссылке, то есть модифицируется исходный массив
a[3] = 42
print(a)

# Если типы элементов не совпадают, то значение "обрезается"
a[3] = 4.2
print(a, a.dtype)

1 4
[ 1  2  3 42]
[1 2 3 4] int32


In [47]:
# Последний элемент
print(a[-1])

a1 = a.reshape(2, 2)
print(a1)

# Индексация производится от строк к столбцам
print(a1[0], a1[1, -1])
print(a1[1, 0], a1[0, 1])

# Так тоже можно
print(a1[1][0], a1[0][1])

4
[[1 2]
 [3 4]]
[1 2] 4
3 2
3 2


Срезы работают аналогично стандартным в Питоне, однако модифицируются исходные данные:

In [56]:
a = np.array([1, 2, 3, 4])
a1 = a.reshape(2, 2)

print(a[::2])

print(a1)
b = a1[:, 1:]
print(b)

# Заменили элемент и в исходных массивах
b[0] = 42
print(a)
print(a1)

[1 3]
[[1 2]
 [3 4]]
[[2]
 [4]]
[ 1 42  3  4]
[[ 1 42]
 [ 3  4]]


Индексация с помощью массивов:

In [72]:
a = np.arange(0, 100, 10)
print(a)
print(a[[0, 1, 2, 3]])
print(a[[-4, -3, -2, -1]])

d = np.identity(5)
idx = np.array([0, 1, 2, 3, 4])
print(d[idx, idx])

[ 0 10 20 30 40 50 60 70 80 90]
[ 0 10 20 30]
[60 70 80 90]
[1. 1. 1. 1. 1.]


Использование булева массива:

In [76]:
bidx = d != 0
print(bidx)
print(d[bidx])
print(d[d == 0])

[[ True False False False False]
 [False  True False False False]
 [False False  True False False]
 [False False False  True False]
 [False False False False  True]]
[1. 1. 1. 1. 1.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Литература

Для продолжения знакомства с функциональным программированием рекомендуются следующие фундаментальные учебники:

1. Абельсон Харольд, Сассман Джеральд Джей. "Структура и Интерпретация Компьютерных Программ".
1. Пирс Бенджамин. "Типы в языках программирования".
1. Окасаки Крис. "Чисто функциональные структуры данных".